# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv(
    "weather.csv", dtype="object", encoding="utf-8")

In [3]:
weather_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Lüderitz,-26.6481,15.1594,66.22,94,4,6.71,NaN,1671047792
1,Klaksvík,62.2266,-6.589,34.72,67,100,28.5,FO,1671047792
2,Vardø,70.3705,31.1107,17.58,79,75,16.11,NO,1671047792
3,Arraial do Cabo,-22.9661,-42.0278,78.76,57,40,12.66,BR,1671047793
4,Bredasdorp,-34.5322,20.0403,69.85,85,47,4.23,ZA,1671047793


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [19]:
locations = weather_data[["Lat", "Lng"]].astype(float)
locations.head()
humidity = weather_data['Humidity'].astype(float)

In [6]:
locations.shape

(537, 2)

In [7]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

In [8]:
fig = gmaps.figure(layout=figure_layout)

In [21]:


heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
hotel_df = weather_data.loc[(weather_data['Max Temp'].astype(float) > 70) & (weather_data['Max Temp'].astype(float)<80)&(weather_data['Wind Speed'].astype(float) < 10)&(weather_data['Cloudiness'].astype(float) == 0)]
hotel_df['Hotel Name'] = ""

/var/folders/3k/6xzjh8pd05z3pk2qlf50c_b80000gn/T/ipykernel_54502/3961563156.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
coords = hotel_df[['Lat','Lng']]

In [24]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel name
179,Poya,-21.35,165.15,77.05,72,0,2.95,NC,1671047692,,Ecole Primaire Publique Carlier Marcel
232,Tombouctou,20.0,-3.0,75.76,13,0,6.04,ML,1671047882,,Sahara desert
237,Peruíbe,-24.32,-46.9983,74.48,57,0,6.91,BR,1671047834,,Brunno Motos
269,Vao,-22.6667,167.4833,74.59,61,0,3.24,NC,1671047896,,Sanctuaire Notre Dame de la Salette
330,Bilma,18.6853,12.9164,71.65,14,0,6.17,NE,1671047918,,Централна джамия Билма
505,Tessalit,20.1986,1.0114,75.0,13,0,7.72,ML,1671047989,,Marisquería el peruano


In [13]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "range": 5000,
    "rankby": "distance",
    "type": "hotel",
    "key": g_key}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    lat = row['Lat']
    lng = row['Lng']
    
    params['location']= f'{lat},{lng}'
    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel name'] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Closest hotel is Ecole Primaire Publique Carlier Marcel.


/var/folders/3k/6xzjh8pd05z3pk2qlf50c_b80000gn/T/ipykernel_54502/2230673024.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.loc[index, 'Hotel name'] = results[0]['name']


------------
Closest hotel is Sahara desert.
------------
Closest hotel is Brunno Motos.
------------
Closest hotel is Sanctuaire Notre Dame de la Salette.
------------
Closest hotel is Централна джамия Билма.
------------
Closest hotel is Marisquería el peruano.
------------


In [14]:
hotel_df.dtypes

City          object
Lat           object
Lng           object
Max Temp      object
Humidity      object
Cloudiness    object
Wind Speed    object
Country       object
Date          object
Hotel Name    object
Hotel name    object
dtype: object

In [15]:
markers = gmaps.marker_layer(coords.astype(float))
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
local = hotel_df[["Lat", "Lng"]]

In [26]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(coords.astype(float),info_box_content=hotel_info)
fig.add_layer(markers)
fig


# Display figure


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…